![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [36]:
-- Write your SQL query here
SELECT * FROM grocery_sales

index  Store_ID       Date  Dept  Weekly_Sales
0            0         1 2010-02-05     1      24924.50
1            1         1 2010-02-05    26      11737.12
2            2         1 2010-02-05    17      13223.76
3            3         1 2010-02-05    45         37.44
4            4         1 2010-02-05    28       1085.29
...        ...       ...        ...   ...           ...
231517  232414        24 2011-05-06     8      49471.07
231518  232415        24 2011-05-06    50       1210.00
231519  232416        24 2011-05-06    87      25893.32
231520  232417        24 2011-05-06    85       1357.83
231521  232418        24 2011-05-06    35       3648.91

[231522 rows x 5 columns]

In [37]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

In [38]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
  # Write your code here  
    raw_data["Date"] = pd.to_datetime(raw_data["Date"], errors='coerce')
    
    # Fill only numeric columns
    numeric_cols = raw_data.select_dtypes(include='number').columns
    raw_data[numeric_cols] = raw_data[numeric_cols].fillna(0)
    
    # Create Month column
    raw_data["Month"] = raw_data["Date"].dt.month
    
    # Filter Weekly_Sales > 10000
    raw_data = raw_data[raw_data["Weekly_Sales"] > 10000]
    
    # Select required columns
    clean_data = raw_data[
        ["Store_ID", "Month", "Dept", "IsHoliday",
         "Weekly_Sales", "CPI", "Unemployment"]
    ]
    
    return clean_data
    
  

In [39]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
clean_data.head(5)

Store_ID  Month  Dept  IsHoliday  Weekly_Sales         CPI  Unemployment
0         1    2.0     1          0      24924.50  211.096358         8.106
1         1    2.0    26          0      11737.12  211.096358         8.106
2         1    2.0    17          0      13223.76  211.096358         8.106
5         1    2.0    79          0      46729.77  211.096358         0.000
6         1    2.0    55          0      21249.31  211.096358         0.000

In [40]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    # Write your code here
    agg_data = (
        clean_data[["Month", "Weekly_Sales"]]
        .groupby("Month")
        .agg(Avg_Sales=("Weekly_Sales", "mean"))
        .reset_index()
        .round(2)
    )
    
    return agg_data
    

In [41]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)


In [42]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    # Write your code here
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)

In [43]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths    
load(clean_data, "clean_data.csv", agg_data, "agg_data.csv")


In [44]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    # Write your code here
    return os.path.exists(file_path)

In [45]:

print(validation("clean_data.csv"))
print(validation("agg_data.csv"))


True
True
